In [54]:
import feather
import re
import math

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import ElasticNetCV
from sklearn import preprocessing


In [69]:
train = feather.read_dataframe('train')
test = feather.read_dataframe('test')

In [68]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    return res

In [70]:
target = pd.read_csv('data\\train.csv',usecols=['card_id','target'])

In [71]:
train = pd.merge(train,target,how='left',on='card_id')

In [72]:
for x in train.columns.values:
    if train[x].isna().sum():
        train[x]
        train[x].fillna(0,inplace=True)
        

In [88]:
(train.agged_all_category_1_sum == 0) | (train.agged_all_category_1_sum == 1)

0          True
1         False
2          True
3         False
4         False
5          True
6          True
7          True
8         False
9          True
10         True
11         True
12         True
13         True
14         True
15        False
16        False
17         True
18         True
19        False
20         True
21        False
22         True
23         True
24        False
25         True
26         True
27        False
28        False
29         True
          ...  
201887    False
201888     True
201889     True
201890     True
201891     True
201892     True
201893    False
201894     True
201895    False
201896     True
201897     True
201898    False
201899    False
201900    False
201901     True
201902    False
201903     True
201904     True
201905    False
201906     True
201907    False
201908     True
201909     True
201910    False
201911     True
201912     True
201913     True
201914    False
201915     True
201916    False
Name: agged_all_category

In [73]:
X_train = train.sample(frac=0.8)
X_valid = train.drop(X_train.index)

In [74]:
y_train = X_train.pop('target')
y_valid = X_valid.pop('target')

In [76]:
regr = ElasticNetCV(l1_ratio=0.6,max_iter=20000,cv=5,n_jobs=4,normalize=True,verbose=True)

In [77]:
to_drop = [x for x in train.columns.values if not (np.issubdtype(train[x],np.number) or np.issubdtype(train[x],np.bool_))]

In [78]:
X_train = X_train.drop(to_drop,axis=1)
X_valid = X_valid.drop(to_drop,axis=1)

In [80]:
X_train = pd.get_dummies(X_train,prefix=['end1','end2','end3'],columns=['feature_1','feature_2','feature_3'])
X_valid = pd.get_dummies(X_valid,prefix=['end1','end2','end3'],columns=['feature_1','feature_2','feature_3'])

In [82]:
regr.fit(X_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=10000, n_alphas=100, n_jobs=4,
       normalize=True, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=True)

In [83]:
print_score(regr)

[3.800481804112205,
 3.828057320007405,
 0.023504117850329687,
 0.02085049134702288]

In [85]:
for x in range(1,10):
    regr = ElasticNetCV(l1_ratio=0.1 * x,max_iter=20000,cv=5,n_jobs=4,normalize=True)
    regr.fit(X_train,y_train)
    print(print_score(regr),x)

[3.8137381119069205, 3.8392074870018233, 0.016680085194192285, 0.01513815224285364] 1
[3.810905823332559, 3.836661986191616, 0.0181400761610645, 0.016443700612284617] 2
[3.8078155243538854, 3.8339839320783837, 0.01973182920531924, 0.017816298828510768] 3
[3.803827295439745, 3.8306978306996484, 0.021784179931903624, 0.019499233397662907] 4
[3.800481804112205, 3.828057320007405, 0.023504117850329687, 0.02085049134702288] 5
[3.7973280793101485, 3.825711476521272, 0.025124081676563104, 0.022050174383127952] 6
[3.7969934161867895, 3.82506646757714, 0.025295908095705477, 0.02237990819264757] 7
[3.797918299184855, 3.8247401333486684, 0.024821007595546174, 0.022546711719904855] 8
[3.797488419897993, 3.823201307345791, 0.02504175248368712, 0.023333080415373852] 9
